In [45]:
import warnings
from functools import lru_cache
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pynverse import inversefunc
from scipy.optimize import brentq

%matplotlib inline

> Replicate figure 3 in [Fuks and Tchelepi, 2020](https://doi.org/10.1615/JMachLearnModelComput.2020033905) using an analytical solution to the Buckley–Leverett equation, non-convex flux problem (equation 21)

<center>$\Large fw(u)=\frac{u^2}{u^2+\frac{(1-u)^2}{M}}$</center>

![](images/equation_21.png)

$\large u^{*}$ = shock location

#### Rankine-Hugoniot Condition:

<center>$\Large f'w(u^{*})=\frac{fw(u^{*}) - fw(u)_{u=0}}{u^{*} - u_{u=0}}$</center>

$\large u(\frac{x}{t}) = (f'w)^{-1} (\frac{x}{t})$

$\large \frac{x}{t}$ = similarity variable

### Expected result:

![](images/nonconvex_flow_results.png)

In [46]:
@lru_cache()
def fw(u, M=1):
    return u**2 / (u ** 2 + ((1 - u) ** 2 / M))

In [47]:
df = pd.DataFrame(data={'u': np.linspace(0, 1, 101)})
df['fw'] = df['u'].map(fw)

In [48]:
px.line(df, x='u', y='fw', title='Non-convex flux').write_image('images/nonconvex_flux.png')

![](images/nonconvex_flux.png)

In [49]:
# f'w = rankine hugoniot condition = rhc
@lru_cache()
def rhc(u):
    return np.divide(fw(u) - fw(0), u)

In [50]:
df['rhc'] = df['u'].map(rhc)

/var/folders/zc/zhbfk_hn05q0qbr8yp3_7t1w0000gn/T/ipykernel_14359/898125747.py:4: RuntimeWarning:

invalid value encountered in true_divide



In [51]:
px.line(df, x='u', y='rhc', title='Rankine-Hugoniot Condition').write_image('images/rankine_hugoniot_cond.png')

![](images/rankine_hugoniot_cond.png)

### Derivative of fw:
<center>$\large f'w=\frac{\partial}{\partial u} \frac{u^2}{u^2 + \frac{(1 - u) ^ 2}{1}}$</center>
<br>
<center>$\large f'w=\frac{2u (1-u)}{(2u^2 - 2u + 1)^2}$</center>

### Inverse of f'w
<center>$\large (f'w)^{-1} = \frac{1}{2}\lgroup 1 + \sqrt{\frac{-2u + \sqrt{4u + 1} - 1}{u} + 1} \rgroup$</center>

In [52]:
@lru_cache()
def fw_deriv(u):
    return np.divide(2 * u * (1 - u), (2 * u ** 2 - 2 * u + 1) ** 2)

@lru_cache()
def root_term(u):
    return np.divide(-2*u + (4 * u + 1) ** (1/2) - 1, u) + 1

@lru_cache()
def inverse_fw_deriv(u):
    return (1 + (root_term(u) ** (1/2))) / 2

In [53]:
def u(x, t, u_star):
    arg = x / t
    if arg > fw_deriv(u_star):
        return 0
    elif fw_deriv(1) >= arg:
        return 1
    return inverse_fw_deriv(arg)

In [54]:
%%time
# tried to make the rankine-hugoniot condition = fw_deriv to find out u*, found the two values below,
# don't know if either of them is right, so I'll just plot them both

u_star_1 = 0.2
u_star_2 = 0.56
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    res = pd.DataFrame()
    for x in np.linspace(0, 1, 101):
        for t in np.linspace(0, 1, 101):
            res = pd.concat([res, pd.DataFrame(data={'u1': u(x, t, u_star_1), 'u2': u(x, t, u_star_2)},
                                               index=pd.MultiIndex.from_arrays([[x], [t]], names=('x', 't')))])

    res = res.reset_index()

CPU times: user 8.97 s, sys: 29.3 ms, total: 9 s
Wall time: 9 s


In [55]:
res.sample(10).sort_index()

,x,t,u1,u2
1671,0.16,0.55,0.893327,0.893327
2210,0.21,0.89,0.909481,0.909481
4174,0.41,0.33,0.000000,0.701046
4283,0.42,0.41,0.000000,0.738658
5903,0.58,0.45,0.000000,0.693056
8414,0.83,0.31,0.000000,0.000000
8628,0.85,0.43,0.000000,0.000000
8930,0.88,0.42,0.000000,0.000000
9028,0.89,0.39,0.000000,0.000000
10049,0.99,0.50,0.000000,0.000000


In [56]:
fig = make_subplots(rows=1, cols=3)
for i, t in enumerate([0.25, 0.5, 0.75]):
    aux = res[res['t'] == t]
    aux_fig = go.Scatter(x=aux['x'], y=aux['u1'], name=f'{t=}')
    fig.update_yaxes(range=[0, 1])
    fig.add_trace(aux_fig, row=1, col=i+1)

fig.update_layout(height=600, width=1000, title_text="Non-convex solution for u* 1")
fig.write_image('images/non_convex_solution_ustart_1.png')

![](images/non_convex_solution_ustart_1.png)

In [58]:
fig = make_subplots(rows=1, cols=3)
for i, t in enumerate([0.25, 0.5, 0.75]):
    aux = res[res['t'] == t]
    aux_fig = go.Scatter(x=aux['x'], y=aux['u2'], name=f'{t=}')
    fig.update_yaxes(range=[0, 1])
    fig.add_trace(aux_fig, row=1, col=i+1)

fig.update_layout(height=600, width=1000, title_text="Non-convex solution for u* 2")
fig.write_image('images/non_convex_solution_ustar_2.png')

![](images/non_convex_solution_ustar_2.png)

neither u* picked is right, the answer seems to lie somewhere in the middle of them. Will try to reverse-engineer value of u*

shock locations:
    
p/ t=0.25, x\~=0.3<br>
p/ t=0.50, x\~= 0.6<br>
p/ t=0.75, x\~= 0.9

<center>$\large \frac{x}{t} = \frac{2u(1 - u)}{(2u^2 - 2u + 1)^2}$</center>

<center>$\large \frac{0.6}{0.5} = \frac{2u(1-u)}{(2u^2 - 2u + 1)^2}$</center>

<center>u = 0.29</center>
<center>u = 0.7</center>

In [59]:
%%time
u_star_1 = 0.29
u_star_2 = 0.7
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    res = pd.DataFrame()
    for x in np.linspace(0, 1, 101):
        for t in np.linspace(0, 1, 101):
            res = pd.concat([res, pd.DataFrame(data={'u1': u(x, t, u_star_1), 'u2': u(x, t, u_star_2)},
                                               index=pd.MultiIndex.from_arrays([[x], [t]], names=('x', 't')))])

    res = res.reset_index()

CPU times: user 8.88 s, sys: 27.6 ms, total: 8.9 s
Wall time: 8.91 s


In [60]:
res.sample(10).sort_index()

,x,t,u1,u2
3330,0.32,0.98,0.883467,0.883467
3446,0.34,0.12,0.000000,0.000000
5626,0.55,0.71,0.783960,0.783960
5824,0.57,0.67,0.769731,0.769731
5880,0.58,0.22,0.000000,0.000000
6326,0.62,0.64,0.748448,0.748448
6346,0.62,0.84,0.790973,0.790973
6932,0.68,0.64,0.732018,0.732018
7119,0.70,0.49,0.000000,0.000000
7942,0.78,0.64,0.000000,0.705109


In [61]:
fig = make_subplots(rows=1, cols=3)
for i, t in enumerate([0.25, 0.5, 0.75]):
    aux = res[res['t'] == t]
    aux_fig = go.Scatter(x=aux['x'], y=aux['u1'], name=f'{t=}')
    fig.update_yaxes(range=[0, 1])
    fig.add_trace(aux_fig, row=1, col=i+1)

fig.update_layout(height=600, width=1000, title_text="Non-convex solution for u* 1")
fig.write_image('images/non_convex_solution_ustar_12.png')

![](images/non_convex_solution_ustar_12.png)

In [63]:
fig = make_subplots(rows=1, cols=3)
for i, t in enumerate([0.25, 0.5, 0.75]):
    aux = res[res['t'] == t]
    aux_fig = go.Scatter(x=aux['x'], y=aux['u2'], name=f'{t=}')
    fig.update_yaxes(range=[0, 1])
    fig.add_trace(aux_fig, row=1, col=i+1)

fig.update_layout(height=600, width=1000, title_text="Non-convex solution for u* 2")
fig.write_image('images/non_convex_solution_ustar_22.png')

![](images/non_convex_solution_ustar_22.png)

In [64]:
aux = res['u1'] - res['u2']
aux[aux > 0]

Series([], dtype: float64)

They are both the same and I don't know why, which is uneasing. But at least they fit figure 3 in [Fuks and Tchelepi, 2020](https://doi.org/10.1615/JMachLearnModelComput.2020033905)

In [65]:
x = res['x'].unique()
y = res['t'].unique()
z = res.pivot('x', 't', 'u1')

In [66]:
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z)])
fig.update_layout(title='Non-convex flux without diffusion term')
fig.write_image('images/non_convex_flux_without_diffusion_term_surface.png')

![](images/non_convex_flux_without_diffusion_term_surface.png)

trying to fit u* using brentq, like suggested in class

In [67]:
def fw_deriv_with_m(u, M):
    return np.divide(-2 * M * (u - 1) * u,
                     ((M + 1) * u ** 2 - 2 * u + 1) ** 2)


def rhc(u, M):
    return np.divide(fw(u, M) - fw_deriv_with_m(0, M), u) - \
           fw_deriv_with_m(u, M)

In [69]:
e = 1e-12
M = 1
u_star = brentq(rhc, e, 1 - e, args=(M))
f'{u_star=}'

'u_star=0.7071067811865476'

pretty close to the 0.7 I got visually, but now I should get perfect results

In [70]:
%%time
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    res = pd.DataFrame()
    for x in np.linspace(0, 1, 101):
        for t in np.linspace(0, 1, 101):
            res = pd.concat([res, pd.DataFrame(data={'u': u(x, t, u_star)},
                                               index=pd.MultiIndex.from_arrays([[x], [t]], names=('x', 't')))])

    res = res.reset_index()

CPU times: user 8.26 s, sys: 23 ms, total: 8.28 s
Wall time: 8.28 s


In [71]:
res.sample(10).sort_index()

,x,t,u
771,0.07,0.64,0.952505
1514,0.14,1.00,0.941183
2795,0.27,0.68,0.865128
3464,0.34,0.30,0.719778
4968,0.49,0.19,0.000000
5015,0.49,0.66,0.790136
6312,0.62,0.50,0.000000
8505,0.84,0.21,0.000000
9762,0.96,0.66,0.000000
10076,0.99,0.77,0.000000


In [72]:
res.to_parquet('Data/buckley_leverett.parquet')

In [73]:
fig = make_subplots(rows=1, cols=3)
for i, t in enumerate([0.25, 0.5, 0.75]):
    aux = res[res['t'] == t]
    aux_fig = go.Scatter(x=aux['x'], y=aux['u'], name=f'{t=}')
    fig.update_yaxes(range=[0, 1])
    fig.add_trace(aux_fig, row=1, col=i+1)

fig.update_layout(height=600, width=1000, title_text="Non-convex solution for correct u*")
fig.write_image('images/nonconvex_solution_correct_star.png')

![](images/nonconvex_solution_correct_star.png)

In [74]:
x = res['x'].unique()
y = res['t'].unique()
z = res.pivot('x', 't', 'u')
fig = go.Figure(data=[go.Surface(x=x, y=y, z=z)])
fig.update_layout(title='Non-convex flux without diffusion term')
fig.write_image('images/nonconvex_flux_without_diffusion_term_surface.png')

![](images/nonconvex_flux_without_diffusion_term_surface.png)